In [1]:
import os 

os.chdir("app/")

In [2]:
from functools import partial

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.trainer import Trainer

from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.data_load.datasets import MemoryMapDataset
from ptls.preprocessing import PandasDataPreprocessor

from ptls.nn import TrxEncoder
from ptls.nn.seq_encoder import RnnSeqEncoder

from ptls.frames import PtlsDataModule
from ptls.frames.coles.coles_module import CoLESModule
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices, NoSplit

from utils.encode import encode_data
from utils.evaluation import bootstrap_eval

## Load data

In [4]:
df = pd.read_parquet("data/preprocessed_new/default_date.parquet")
df.head()

,user_id,mcc_code,amount,timestamp,holiday_target,weekend_target,global_target,default_target,time_delta
0,69,5,-342.89792,2021-03-05 02:52:36,0,0,0,0,0
1,69,21,-1251.88120,2021-03-05 09:43:28,0,0,0,0,24652
2,69,12,-87.30924,2021-03-05 11:17:23,0,0,0,0,5635
3,69,6,-1822.17700,2021-03-05 13:41:03,0,0,0,0,8620
4,69,18,-427.12363,2021-03-05 19:14:23,0,0,0,0,20000


## Prepare dataset and dataloaders

In [5]:
preprocessor = PandasDataPreprocessor(
    col_id="user_id",
    col_event_time="timestamp",
    event_time_transformation="dt_to_timestamp",
    cols_category=["mcc_code"],
    cols_first_item=["global_target"]
)

In [6]:
data = preprocessor.fit_transform(df)

In [7]:
y = np.array([data[i]["global_target"] for i in range(len(data))])

In [8]:
val_size = 0.1
test_size = 0.1

train, val_test, y_train, y_val_test = train_test_split(data, y, stratify=y, test_size=test_size+val_size, random_state=42)
val, test, y_val, y_test = train_test_split(val_test, y_val_test, stratify=y_val_test, test_size=test_size/(test_size+val_size), random_state=42)

In [9]:
train_ds = ColesDataset(
    data=MemoryMapDataset(train, [SeqLenFilter(min_seq_len=15)]),
    splitter=SampleSlices(5, 15, 150),
)
val_ds = ColesDataset(
    data=MemoryMapDataset(val, [SeqLenFilter(min_seq_len=15)]),
    splitter=SampleSlices(5, 15, 150),
)
test_ds = MemoryMapDataset(test, [SeqLenFilter(min_seq_len=15)])

In [10]:
datamodule = PtlsDataModule(
    train_data=train_ds,
    valid_data=val_ds,
    train_batch_size=128,
    valid_batch_size=128,
    train_num_workers=8,
    valid_num_workers=8
)

# CoLES

## Model training

In [15]:
trx_encoder = TrxEncoder(
    embeddings={
        "mcc_code": {"in": 345, "out": 16}
    },
    numeric_values={
        "amount": "identity",
    },
    use_batch_norm_with_lens=True,
    norm_embeddings=False,
    embeddings_noise=0.003
)

seq_encoder = RnnSeqEncoder(
    trx_encoder,
    hidden_size=800,
    type="gru",
    bidir=False,
    trainable_starter="static",
)

In [16]:
lr_scheduler_partial = partial(torch.optim.lr_scheduler.ReduceLROnPlateau, factor=.9025, patience=5, mode="max")
optimizer_partial = partial(torch.optim.Adam, lr=4e-3)

model = CoLESModule(
    seq_encoder,
    optimizer_partial=optimizer_partial,
    lr_scheduler_partial=lr_scheduler_partial
)

In [ ]:
checkpoint = ModelCheckpoint(
    monitor="recall_top_k", 
    mode="max"
)

trainer = Trainer(
    max_epochs=50,
    devices=[1],
    accelerator="gpu",
    callbacks=[checkpoint]
)

trainer.fit(model, datamodule)

In [18]:
model.load_state_dict(torch.load(checkpoint.best_model_path)["state_dict"])

<All keys matched successfully>

In [26]:
# torch.save(model.seq_encoder.state_dict(), "coles_default.pth")

## Model evaluation

In [27]:
seq_encoder.load_state_dict(torch.load("checkpoints/default/coles/coles_default.pth"))

<All keys matched successfully>

In [28]:
train_val_ds = MemoryMapDataset(train + val, [SeqLenFilter(min_seq_len=15)])

In [29]:
X_train, y_train = encode_data(model.seq_encoder, train_val_ds)
X_test, y_test = encode_data(model.seq_encoder, test_ds)

print("Train size:", len(y_train))
print("Test size:", len(y_test))

Train size: 6372
Test size: 708


In [30]:
from sklearn.metrics import roc_auc_score, average_precision_score
from lightgbm import LGBMClassifier


lgbm = LGBMClassifier(
    n_estimators=500,
    boosting_type="gbdt",
    subsample=0.5,
    subsample_freq=1,
    learning_rate=0.02,
    feature_fraction=0.75,
    max_depth=6,
    lambda_l1=1,
    lambda_l2=1,
    min_data_in_leaf=50,
    random_state=42,
    n_jobs=8,
    verbose=-1
)

lgbm.fit(X_train, y_train)

LGBMClassifier(feature_fraction=0.75, lambda_l1=1, lambda_l2=1,
               learning_rate=0.02, max_depth=6, min_data_in_leaf=50,
               n_estimators=500, n_jobs=8, random_state=42, subsample=0.5,
               subsample_freq=1, verbose=-1)

In [31]:
y_pred = lgbm.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred), average_precision_score(y_test, y_pred)

(0.5772614482291902, 0.05325062762062647)

# CoLES with time features

## Model training

In [32]:
trx_encoder = TrxEncoder(
    embeddings={
        "mcc_code": {"in": 345, "out": 24}
    },
    numeric_values={
        "amount": "identity",
        "event_time": "identity",
        "time_delta": "identity",
    },
    use_batch_norm_with_lens=True,
    norm_embeddings=False,
    embeddings_noise=0.003
)

seq_encoder = RnnSeqEncoder(
    trx_encoder,
    hidden_size=800,
    type="gru",
    bidir=False,
    trainable_starter="static",
)

In [33]:
lr_scheduler_partial = partial(torch.optim.lr_scheduler.ReduceLROnPlateau, factor=.9025, patience=5, mode="max")
optimizer_partial = partial(torch.optim.Adam, lr=4e-3)

model = CoLESModule(
    seq_encoder,
    optimizer_partial=optimizer_partial,
    lr_scheduler_partial=lr_scheduler_partial
)

In [ ]:
checkpoint = ModelCheckpoint(
    monitor="recall_top_k", 
    mode="max"
)

trainer = Trainer(
    max_epochs=50,
    devices=[1],
    accelerator="gpu",
    callbacks=[checkpoint]
)

trainer.fit(model, datamodule)

In [35]:
model.load_state_dict(torch.load(checkpoint.best_model_path)["state_dict"])

<All keys matched successfully>

In [42]:
# torch.save(model.seq_encoder.state_dict(), "coles_default_date.pth")

## Model evaluation

In [37]:
train_val_ds = MemoryMapDataset(train + val, [SeqLenFilter(min_seq_len=15)])

In [38]:
X_train, y_train = encode_data(model.seq_encoder, train_val_ds)
X_test, y_test = encode_data(model.seq_encoder, test_ds)

print("Train size:", len(y_train))
print("Test size:", len(y_test))

Train size: 6372
Test size: 708


In [39]:
# results = bootstrap_eval(X_train, X_test, y_train, y_test, n_runs=10)

In [40]:
from sklearn.metrics import roc_auc_score, average_precision_score
from lightgbm import LGBMClassifier


lgbm = LGBMClassifier(
    n_estimators=500,
    boosting_type="gbdt",
    subsample=0.5,
    subsample_freq=1,
    learning_rate=0.02,
    feature_fraction=0.75,
    max_depth=6,
    lambda_l1=1,
    lambda_l2=1,
    min_data_in_leaf=50,
    random_state=42,
    n_jobs=8,
    verbose=-1
)

lgbm.fit(X_train, y_train)

LGBMClassifier(feature_fraction=0.75, lambda_l1=1, lambda_l2=1,
               learning_rate=0.02, max_depth=6, min_data_in_leaf=50,
               n_estimators=500, n_jobs=8, random_state=42, subsample=0.5,
               subsample_freq=1, verbose=-1)

In [41]:
y_pred = lgbm.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred), average_precision_score(y_test, y_pred)

(0.6007782540040605, 0.06407901614867755)